In [90]:
import os, sys
import numpy as np
import pandas as pd
import scanpy as sc

from tqdm import tqdm

import time
import logging

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

sys.path.append("..")

import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact

from functools import partial
from masking import data_mask
from data.multi_file_dataset import MultiScRNADataset as scRNADataset
from model.performer.performer import PerformerLM

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [48]:
DATA_PREFIX = "/mnt/data/01_repos/scFoundationModel/data/transforms/root_3932_genes"

In [49]:
# data_path = f"{os.getenv('HOME')}/data/scrna_seq_arabidopsis/transforms/root_3932_genes"
test_dataset = scRNADataset(f"{DATA_PREFIX}/test")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
ckpt = torch.load("../checkpoints/02cfdb5ecf904c9fb8c734f82c56171d/performer_model_13.pth", map_location="cpu")
model_weights = { k.replace("_orig_mod.", ""): v for k,v in ckpt['model_state_dict'].items() }

N_GENES = 3932

model = PerformerLM(**{
    "num_tokens": 7,
    "dim": 200,
    "depth": 4,
    "max_seq_len": N_GENES + 1,
    "heads": 10,
    "local_attn_heads": 0,
    "g2v_position_emb": None
}).to(device)

model.load_state_dict(model_weights)

/home/rbonazzola/repos/scFoundationModel/notebooks/../model/performer/performer.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2485.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)


<All keys matched successfully>

In [ ]:
# test_dataset[0].unsqueeze(0)

In [ ]:
mask = partial(data_mask, mask_prob=1.00)

x = test_dataset[0].unsqueeze(0)
pd.Series(mask(x)[0][0]).value_counts()

0    3758
6     162
3       8
2       4
Name: count, dtype: int64

In [ ]:
@interact
def show_conf_matrix(i=widgets.IntSlider(min=0, max=10000), prob_mask=widgets.FloatSlider(min=0, max=1, value=0, step=0.05)):

    mask = partial(data_mask, mask_prob=prob_mask)
    
    x = test_dataset[i].unsqueeze(0).to(device)  
    x_masked, _ = mask(x)

    true_labels = test_dataset[i].detach()
    with torch.no_grad():
        pred_labels_no_masking = torch.softmax(model(x).squeeze(0), 1).argmax(axis=1)
        pred_labels_masking    = torch.softmax(model(x_masked).squeeze(0), 1).argmax(axis=1)
    
    y_true            = true_labels.cpu().numpy()
    y_pred_no_masking = pred_labels_no_masking.cpu().numpy()
    y_pred_masking    = pred_labels_masking.cpu().numpy()
    
    bins = range(1, 7)
    cm_no_mask = confusion_matrix(y_true, y_pred_no_masking, labels=bins)
    cm_mask    = confusion_matrix(y_true, y_pred_masking   , labels=bins)
    
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_mask, display_labels=[f"B{i}" for i in bins])
    disp.plot(cmap="Blues", values_format="d")
    plt.title("Confusion Matrix - scBERT")
    plt.show()

interactive(children=(IntSlider(value=0, description='i', max=10000), FloatSlider(value=0.0, description='prob…

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32, num_workers=16)

In [ ]:
mask = partial(data_mask, mask_prob=0.15)    
    
for batch in tqdm(test_dataloader):
        
    x = batch # test_dataset[i].unsqueeze(0).to(device)  
    x = x.to(device)
    x_masked, _ = mask(x)

    # true_labels = test_dataset[i].detach()
    with torch.no_grad():
        pred_labels_no_masking = torch.softmax(model(x).squeeze(0), 1).argmax(axis=1)
        # pred_labels_masking    = torch.softmax(model(x_masked, return_encodings=True).squeeze(0), 1).argmax(axis=1)
        pred_labels_masking    = model(x_masked, return_encodings=True)
    
    # y_true            = true_labels.cpu().numpy()
    # y_pred_no_masking = pred_labels_no_masking.cpu().numpy()
    # y_pred_masking    = pred_labels_masking.cpu().numpy()
    
    # bins = range(1, 7)
    # cm_no_mask = confusion_matrix(y_true, y_pred_no_masking, labels=bins)
    # cm_mask    = confusion_matrix(y_true, y_pred_masking   , labels=bins)
    
    # disp = ConfusionMatrixDisplay(confusion_matrix=cm_mask, display_labels=[f"B{i}" for i in bins])
    # disp.plot(cmap="Blues", values_format="d")
    # plt.title("Confusion Matrix - scBERT")
    # plt.show()

  0%|          | 10/3684 [00:06<39:31,  1.55it/s]


KeyboardInterrupt: 

In [ ]:
folder_path = "/home/rbonazzola/data/scrna_seq_arabidopsis/transforms/root_3932_genes/"

In [57]:
N_CLASSES = 7

class SCDataset(torch.utils.data.Dataset):
    
    def __init__(self, data_path):
        
        h5ad_files = sorted([f for f in os.listdir(DATA_PREFIX) if f.endswith('.h5ad')])
        h5ad_files

        file_paths = [os.path.join(DATA_PREFIX, f) for f in h5ad_files]

        # Find common genes
        gene_sets = []
        for path in file_paths:
            adata = sc.read_h5ad(path, backed='r')
            gene_sets.append(set(adata.var_names))
            adata.file.close()
        common_genes = set.intersection(*gene_sets)
 
        first_adata = sc.read_h5ad(file_paths[0], backed='r')
        selected_genes = [g for g in first_adata.var_names if g in common_genes][:4000]
        first_adata.file.close()

        adata = sc.read_h5ad(data_path)
        adata = adata[:, selected_genes]
        X = adata.X.toarray()        
        
        markers_df = pd.read_csv("../arabidopsis_thaliana.marker_fd.csv.gz", compression="gzip")
        markers_df = markers_df.query("tissue == 'Root'")
        markers_df = markers_df[markers_df['gene'].isin(adata.var_names)]
        
        celltype_scores = {}
        for cell_type in markers_df['clusterName'].unique():        
            sub = markers_df[markers_df['clusterName'] == cell_type]
            expr_matrix = adata[:, sub['gene']].X.toarray()
            weights = sub['avg_log2FC'].values
            weighted_score = expr_matrix @ weights
            celltype_scores[cell_type] = weighted_score
        
        
        score_df = pd.DataFrame(celltype_scores, index=adata.obs_names)
        adata.obs["predicted_celltype"] = score_df.idxmax(axis=1)
        
        self.data = torch.Tensor(np.clip(X, a_min=0, a_max=N_CLASSES - 2).astype(np.int64))
        
        self.cell_type_labels = pd.get_dummies(adata.obs["predicted_celltype"]).astype(float)
        self.label_names = self.cell_type_labels.columns
        self.cell_type_labels = torch.Tensor(self.cell_type_labels.values)

    def __getitem__(self, index):
        
        return self.cell_type_labels[index], self.data[index].long()
    
    def __len__(self):
        return len(self.data)



In [63]:
sc.read_h5ad(path, backed='r')

NameError: name 'path' is not defined

In [60]:
data_path = f"{DATA_PREFIX}/SRP148288_transformed_subset.h5ad"
dataset = SCDataset(data_path=data_path)

/tmp/ipykernel_289561/2486894387.py:37: RuntimeWarning: invalid value encountered in matmul
  weighted_score = expr_matrix @ weights
/tmp/ipykernel_289561/2486894387.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["predicted_celltype"] = score_df.idxmax(axis=1)


In [62]:
dataset[0]

(tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0, 0, 4,  ..., 0, 0, 0]))

In [ ]:
dataset.cell_type_labels.sum(axis=0)

tensor([2655.,   31.,   22.,  968.,  117., 4516.,  332.,  114., 1629.,   28.,
        2207., 1283.,   12., 3162., 1125., 6108.,   60.])

In [ ]:
dataset.label_names

Index(['Columella root cap', 'Companion cell', 'G1/G0 phase', 'G2/M phase',
       'Lateral root cap', 'Metaxylem', 'Non-hair', 'Phloem/Pericycle',
       'Protoxylem', 'Root cap', 'Root cortex', 'Root endodermis',
       'Root epidermis', 'Root hair', 'Root stele', 'S phase',
       'Sieve element'],
      dtype='object')

Regresión logística a partir de expresión

In [ ]:
X_expr = dataset.data.numpy()
y = dataset.cell_type_labels.numpy()
X_train, X_test, y_train, y_test = train_test_split(X_expr, y, test_size=0.2, random_state=42)
y_train = y_train.argmax(axis=1)

clf_expr = LogisticRegression(max_iter=12,  multi_class='multinomial', solver='lbfgs')
clf_expr.fit(X_train, y_train)

/home/rbonazzola/miniconda/envs/scrna/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/rbonazzola/miniconda/envs/scrna/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=12, multi_class='multinomial')

In [ ]:
y_pred_expr = clf_expr.predict(X_test)
print(classification_report(y_test.argmax(axis=1), y_pred_expr, target_names=dataset.label_names))

                    precision    recall  f1-score   support

Columella root cap       0.78      0.86      0.82       551
    Companion cell       0.00      0.00      0.00         2
       G1/G0 phase       0.00      0.00      0.00         6
        G2/M phase       0.85      0.87      0.86       191
  Lateral root cap       0.47      0.27      0.34        26
         Metaxylem       0.88      0.84      0.86       904
          Non-hair       0.36      0.23      0.28        61
  Phloem/Pericycle       1.00      0.11      0.19        28
        Protoxylem       0.77      0.66      0.71       304
          Root cap       0.00      0.00      0.00         7
       Root cortex       0.68      0.80      0.74       413
   Root endodermis       0.65      0.70      0.67       266
    Root epidermis       0.00      0.00      0.00         4
         Root hair       0.77      0.79      0.78       635
        Root stele       0.78      0.78      0.78       232
           S phase       0.91      0.91

/home/rbonazzola/miniconda/envs/scrna/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rbonazzola/miniconda/envs/scrna/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rbonazzola/miniconda/envs/scrna/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [ ]:
# mask = partial(data_mask, mask_prob=0.15)    
embeddings = []
labels = []

dataloader = DataLoader(dataset, batch_size=8, num_workers=8, shuffle=False)

for batch in tqdm(dataloader):
        
    y, x = batch # test_dataset[i].unsqueeze(0).to(device)  
    x = x.to(device)
    labels.append(y) # x = y.to(device)
    # x_masked, _ = mask(x)

    # true_labels = test_dataset[i].detach()
    # with torch.no_grad():
    #     # pred_labels_no_masking = torch.softmax(model(x).squeeze(0), 1).argmax(axis=1)
    #     # pred_labels_masking    = torch.softmax(model(x_masked, return_encodings=True).squeeze(0), 1).argmax(axis=1)        
    #     output    = model(x, return_encodings=True)[:,0,:]
    #     embedding = output.to("cpu")
    #     # print(embedding.shape)
    #     embeddings.append(embedding)
    
    # y_true            = true_labels.cpu().numpy()
    # y_pred_no_masking = pred_labels_no_masking.cpu().numpy()
    # y_pred_masking    = pred_labels_masking.cpu().numpy()
    
    # bins = range(1, 7)
    # cm_no_mask = confusion_matrix(y_true, y_pred_no_masking, labels=bins)
    # cm_mask    = confusion_matrix(y_true, y_pred_masking   , labels=bins)
    
    # disp = ConfusionMatrixDisplay(confusion_matrix=cm_mask, display_labels=[f"B{i}" for i in bins])
    # disp.plot(cmap="Blues", values_format="d")
    # plt.title("Confusion Matrix - scBERT")
    # plt.show()

    # np.save("embeddings_SRP148288_root.npy", torch.concat(embeddings).numpy())

100%|██████████| 3047/3047 [00:01<00:00, 1837.56it/s]


In [ ]:
one_hot_labels = torch.concat(labels)

In [ ]:
np.save("labels.npy", one_hot_labels.numpy())

In [ ]:
embeddings = np.load("embeddings_SRP148288_root.npy", allow_pickle=True)
labels = np.load("labels.npy", allow_pickle=True) 

In [ ]:
X_emb = torch.Tensor(embeddings)
one_hot_labels = torch.concat(labels)

TypeError: concat(): argument 'tensors' (position 1) must be tuple of Tensors, not numpy.ndarray

In [ ]:
# X_expr = dataset.data.numpy()
# y = dataset.cell_type_labels.numpy()
X_train, X_test, y_train, y_test = train_test_split(X_emb.numpy(), one_hot_labels, test_size=0.2, random_state=42)
y_train = y_train.argmax(axis=1)

## Determinar condiciones a partir de barcodes

In [ ]:
datapath = "/home/rodrigo/Descargas/SRP148288/"

infer_condition = lambda file: "uninduced" if "uninduced" in file else "induced"
data_and_conditions = [ (os.path.join(datapath, file), infer_condition(file)) for file in os.listdir(datapath) ]

In [43]:
cell_ids_per_condition = {"induced": [], "uninduced": []}
for i in range(6):
    condition = data_and_conditions[i][1]
    file = data_and_conditions[i][0]
    (cell_ids := pd.read_csv(file, sep='\t').columns.to_list())
    cell_ids_per_condition[condition].extend(cell_ids)

cell_ids_per_condition['induced'] = set(cell_ids_per_condition['induced'])
cell_ids_per_condition['induced'].remove("GENE")

cell_ids_per_condition['uninduced'] = set(cell_ids_per_condition['uninduced'])
cell_ids_per_condition['uninduced'].remove("GENE")

In [ ]:
clf_expr = LogisticRegression(max_iter=10000,  multi_class='multinomial', solver='lbfgs')
clf_expr.fit(X_train, y_train)

y_pred_expr = clf_expr.predict(X_test)

/home/rbonazzola/miniconda/envs/scrna/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
from sklearn.manifold import TSNE

embeddings_tsne = TSNE(n_components=2).fit_transform(embeddings)
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], c=y, cmap="tab20")

: 

___

In [146]:
data_path = f"{DATA_PREFIX}/SRP148288_transformed_subset.h5ad"

adata = sc.read_h5ad(data_path)
conditions = torch.Tensor(adata.obs.Condition == "Normal").unsqueeze(1)

embeddings = np.load("embeddings_SRP148288_root.npy", allow_pickle=True)
labels = np.load("labels.npy", allow_pickle=True) 
X_emb = torch.Tensor(embeddings)

/tmp/ipykernel_289561/2241885972.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conditions = torch.Tensor(adata.obs.Condition == "Normal").unsqueeze(1)


In [149]:
X_train, X_test, y_train, y_test = train_test_split(X_emb.numpy(), conditions.numpy(), test_size=0.15, random_state=42)

clf_expr = LogisticRegression(max_iter=10000,  multi_class='multinomial', solver='lbfgs')
clf_expr.fit(X_train, y_train)

y_pred_emb = clf_expr.predict(X_test)

print(classification_report(
    y_test, y_pred_emb,
    target_names=["normal", "treated with estradiol"])
)

/home/rodrigo/miniconda/envs/scRNA/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rodrigo/miniconda/envs/scRNA/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


                        precision    recall  f1-score   support

                normal       0.68      0.90      0.77      2038
treated with estradiol       0.78      0.47      0.59      1618

              accuracy                           0.71      3656
             macro avg       0.73      0.69      0.68      3656
          weighted avg       0.73      0.71      0.69      3656



In [ ]:
X_expr = adata.X.todense()
X_expr = np.asarray(X_expr)
X_train, X_test, y_train, y_test = train_test_split(X_expr, conditions.numpy().astype(int), test_size=0.15, random_state=42)
# y_train = y_train.argmax(axis=1)

clf_expr = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
clf_expr.fit(X_train, y_train)
y_pred_expr = clf_expr.predict(X_test)

print(classification_report(
    y_test, y_pred_expr,
    target_names=["normal", "treated with estradiol"])
)

/home/rodrigo/miniconda/envs/scRNA/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rodrigo/miniconda/envs/scRNA/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


                        precision    recall  f1-score   support

                normal       0.81      0.79      0.80      2709
treated with estradiol       0.75      0.77      0.76      2165

              accuracy                           0.78      4874
             macro avg       0.78      0.78      0.78      4874
          weighted avg       0.78      0.78      0.78      4874

